### [AI vs Human 텍스트 판별 해커톤 -월간 데이콘 쇼츠](https://dacon.io/competitions/official/236178/overview/description)

4일 후의 프로젝트를 위해 준비된 이 데이터셋에는 인간이 작성한 리뷰와 인공지능이 작성한 리뷰가 섞여 있습니다.

하지만 어떤 리뷰가 인간에 의해 작성되었는지를 나타내는 레이블 대부분이 사라져버렸습니다.

여러분의 임무는 일부 레이블이 남아있는 학습용 데이터셋을 활용하여,

테스트 데이터셋의 네 개 리뷰 중 어떤 것이 실제 인간에 의해 작성된 것인지 정확하게 예측하는 것입니다!

당신의 통찰력을 활용하여 테스트 데이터셋의 'label' 필드를 복구해주세요!

In [1]:
import pandas as pd
import numpy as np
import random
import os

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

from tqdm import tqdm

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertModel, BertTokenizer

import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def set_seed(seed=42):
    np.random.seed(seed)  # 이 부분이 pandas의 sample 함수에도 영향을 줍니다.
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed()

In [4]:
train_data = pd.read_csv('/content/drive/MyDrive/ESAA/OB/data/ai_vs_human_text/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/ESAA/OB/data/ai_vs_human_text/test.csv')
print(train_data.shape, test_data.shape)

(50, 6) (1100, 5)


In [13]:
print(train_data.iloc[0])

id                                                   TRAIN_000
sentence1    직원들 마음에 들지 않는다는 것은 알겠지만, 가지 말아야 할까? 인터넷에서 싸게 살...
sentence2    직원들 진짜 싸가지 없어요 ㅋㅋㅋㅋ 가지 마숑  인터넷이 더 싼거 알면서도 이것저것...
sentence3    직원들 정말 싸가지 없네요 ㅋㅋㅋㅋ 인터넷에서 더 싸게 구입할 수 있다는 걸 알면서...
sentence4    직원들의 태도가 정말 별로였어요 ㅋㅋㅋㅋ 가볼만한 가게라는 소문을 듣고 인터넷으로 ...
label                                                        2
Name: 0, dtype: object


In [23]:
a = train_data.iloc[0][1]
b = train_data.iloc[0][2]
c = train_data.iloc[0][3]
d = train_data.iloc[0][4]

df = pd.DataFrame({'문장1': [a], '문장2': [b], '문장3': [c], '문장4': [d]}).T
df.columns = ['Sentence']
df

,Sentence
문장1,"직원들 마음에 들지 않는다는 것은 알겠지만, 가지 말아야 할까? 인터넷에서 싸게 살 수 있는 것도 알면서 나만의 질문을 하려고 하니까요. 그런데 직원들은 걸어서 대화하거나, 아무 대답이 없는 경우가 많아요. 그래서 결국 인터넷에서 직접 찾아보고 구매하려고 생각했어요. 가지 마세요. 감정적으로 상하고 시간만 낭비하게 될 거에요."
문장2,직원들 진짜 싸가지 없어요 ㅋㅋㅋㅋ 가지 마숑 인터넷이 더 싼거 알면서도 이것저것 좀 물어보려는데 돈 되는거 사는거 물어보는 질문아니면. 직원이 아예 걸어가면서 말하거나... (고객은 뒤에 따라가면서 들음) or 모르쇠로 일관 (무성의 말투로 ..) 그냥 인터넷에서 물어보고 인터넷으로 구매해야겠습니당. 가지마세요. 감정상하고 시간 아까움
문장3,"직원들 정말 싸가지 없네요 ㅋㅋㅋㅋ 인터넷에서 더 싸게 구입할 수 있다는 걸 알면서도 여러 가지를 물어보려고 했는데, 돈을 벌 수 있는 질문이 아니면 직원은 걸어가며 말하던지, 모르는 척하더라고요. (고객은 뒤를 따라가며 듣고 있음) 그냥 인터넷에서 문의하고 구매하면 되겠네요. 가지 마세요. 감정이 상하고 시간만 아까워요."
문장4,"직원들의 태도가 정말 별로였어요 ㅋㅋㅋㅋ 가볼만한 가게라는 소문을 듣고 인터넷으로 더 싼 가격을 알고 있었는데, 여러 가지 질문을 하려고 했는데도 돈을 벌어야하는 질문이 아니면 직원들은 관심을 가지지 않았어요. 직원이 걸어가면서 말하거나... (고객은 직원 뒤를 따라가며 듣음) 또는 전혀 알지 못하는척 일관했어요 (무성의한 어조로...) 그냥 인터넷에서 질문하고 인터넷으로 구매해야겠어요. 가지마세요. 감정적으로 피해를 입히고 시간을 낭비할 뿐입니다."


In [25]:
styled_df = df.style.set_properties(**{'text-align': 'left'}) \
                    .set_table_styles({
                        'Column1': [{'selector': '', 'props': [('min-width', '200px')]}],
                        'Column2': [{'selector': '', 'props': [('min-width', '500px')]}]
                    })

styled_df

,Sentence
문장1,"직원들 마음에 들지 않는다는 것은 알겠지만, 가지 말아야 할까? 인터넷에서 싸게 살 수 있는 것도 알면서 나만의 질문을 하려고 하니까요. 그런데 직원들은 걸어서 대화하거나, 아무 대답이 없는 경우가 많아요. 그래서 결국 인터넷에서 직접 찾아보고 구매하려고 생각했어요. 가지 마세요. 감정적으로 상하고 시간만 낭비하게 될 거에요."
문장2,직원들 진짜 싸가지 없어요 ㅋㅋㅋㅋ 가지 마숑 인터넷이 더 싼거 알면서도 이것저것 좀 물어보려는데 돈 되는거 사는거 물어보는 질문아니면. 직원이 아예 걸어가면서 말하거나... (고객은 뒤에 따라가면서 들음) or 모르쇠로 일관 (무성의 말투로 ..) 그냥 인터넷에서 물어보고 인터넷으로 구매해야겠습니당. 가지마세요. 감정상하고 시간 아까움
문장3,"직원들 정말 싸가지 없네요 ㅋㅋㅋㅋ 인터넷에서 더 싸게 구입할 수 있다는 걸 알면서도 여러 가지를 물어보려고 했는데, 돈을 벌 수 있는 질문이 아니면 직원은 걸어가며 말하던지, 모르는 척하더라고요. (고객은 뒤를 따라가며 듣고 있음) 그냥 인터넷에서 문의하고 구매하면 되겠네요. 가지 마세요. 감정이 상하고 시간만 아까워요."
문장4,"직원들의 태도가 정말 별로였어요 ㅋㅋㅋㅋ 가볼만한 가게라는 소문을 듣고 인터넷으로 더 싼 가격을 알고 있었는데, 여러 가지 질문을 하려고 했는데도 돈을 벌어야하는 질문이 아니면 직원들은 관심을 가지지 않았어요. 직원이 걸어가면서 말하거나... (고객은 직원 뒤를 따라가며 듣음) 또는 전혀 알지 못하는척 일관했어요 (무성의한 어조로...) 그냥 인터넷에서 질문하고 인터넷으로 구매해야겠어요. 가지마세요. 감정적으로 피해를 입히고 시간을 낭비할 뿐입니다."


In [26]:
pd.set_option('display.max_colwidth', None)
df = df.style.set_properties(**{'text-align': 'left'})

In [27]:
df

,Sentence
문장1,"직원들 마음에 들지 않는다는 것은 알겠지만, 가지 말아야 할까? 인터넷에서 싸게 살 수 있는 것도 알면서 나만의 질문을 하려고 하니까요. 그런데 직원들은 걸어서 대화하거나, 아무 대답이 없는 경우가 많아요. 그래서 결국 인터넷에서 직접 찾아보고 구매하려고 생각했어요. 가지 마세요. 감정적으로 상하고 시간만 낭비하게 될 거에요."
문장2,직원들 진짜 싸가지 없어요 ㅋㅋㅋㅋ 가지 마숑 인터넷이 더 싼거 알면서도 이것저것 좀 물어보려는데 돈 되는거 사는거 물어보는 질문아니면. 직원이 아예 걸어가면서 말하거나... (고객은 뒤에 따라가면서 들음) or 모르쇠로 일관 (무성의 말투로 ..) 그냥 인터넷에서 물어보고 인터넷으로 구매해야겠습니당. 가지마세요. 감정상하고 시간 아까움
문장3,"직원들 정말 싸가지 없네요 ㅋㅋㅋㅋ 인터넷에서 더 싸게 구입할 수 있다는 걸 알면서도 여러 가지를 물어보려고 했는데, 돈을 벌 수 있는 질문이 아니면 직원은 걸어가며 말하던지, 모르는 척하더라고요. (고객은 뒤를 따라가며 듣고 있음) 그냥 인터넷에서 문의하고 구매하면 되겠네요. 가지 마세요. 감정이 상하고 시간만 아까워요."
문장4,"직원들의 태도가 정말 별로였어요 ㅋㅋㅋㅋ 가볼만한 가게라는 소문을 듣고 인터넷으로 더 싼 가격을 알고 있었는데, 여러 가지 질문을 하려고 했는데도 돈을 벌어야하는 질문이 아니면 직원들은 관심을 가지지 않았어요. 직원이 걸어가면서 말하거나... (고객은 직원 뒤를 따라가며 듣음) 또는 전혀 알지 못하는척 일관했어요 (무성의한 어조로...) 그냥 인터넷에서 질문하고 인터넷으로 구매해야겠어요. 가지마세요. 감정적으로 피해를 입히고 시간을 낭비할 뿐입니다."


In [14]:
print('문장 1 :', train_data.iloc[0][1])
print('문장 2 :', train_data.iloc[0][2]) # 정답
print('문장 3 :', train_data.iloc[0][3])
print('문장 4 :', train_data.iloc[0][4])

문장 1 : 직원들 마음에 들지 않는다는 것은 알겠지만, 가지 말아야 할까? 인터넷에서 싸게 살 수 있는 것도 알면서 나만의 질문을 하려고 하니까요. 그런데 직원들은 걸어서 대화하거나, 아무 대답이 없는 경우가 많아요. 그래서 결국 인터넷에서 직접 찾아보고 구매하려고 생각했어요. 가지 마세요. 감정적으로 상하고 시간만 낭비하게 될 거에요.
문장 2 : 직원들 진짜 싸가지 없어요 ㅋㅋㅋㅋ 가지 마숑  인터넷이 더 싼거 알면서도 이것저것 좀 물어보려는데  돈 되는거 사는거 물어보는 질문아니면.   직원이 아예 걸어가면서 말하거나... (고객은 뒤에 따라가면서 들음)  or  모르쇠로 일관 (무성의 말투로 ..)   그냥 인터넷에서 물어보고 인터넷으로 구매해야겠습니당.  가지마세요. 감정상하고 시간 아까움
문장 3 : 직원들 정말 싸가지 없네요 ㅋㅋㅋㅋ 인터넷에서 더 싸게 구입할 수 있다는 걸 알면서도 여러 가지를 물어보려고 했는데, 돈을 벌 수 있는 질문이 아니면 직원은 걸어가며 말하던지, 모르는 척하더라고요. (고객은 뒤를 따라가며 듣고 있음) 그냥 인터넷에서 문의하고 구매하면 되겠네요. 가지 마세요. 감정이 상하고 시간만 아까워요.
문장 4 : 직원들의 태도가 정말 별로였어요 ㅋㅋㅋㅋ 가볼만한 가게라는 소문을 듣고 인터넷으로 더 싼 가격을 알고 있었는데, 여러 가지 질문을 하려고 했는데도 돈을 벌어야하는 질문이 아니면 직원들은 관심을 가지지 않았어요. 직원이 걸어가면서 말하거나... (고객은 직원 뒤를 따라가며 듣음) 또는 전혀 알지 못하는척 일관했어요 (무성의한 어조로...) 그냥 인터넷에서 질문하고 인터넷으로 구매해야겠어요. 가지마세요. 감정적으로 피해를 입히고 시간을 낭비할 뿐입니다.


In [ ]:
## 정답 ##
print(train_data.iloc[0][5])

2


In [ ]:
# Melt the training data
train_data_melted = train_data.melt(id_vars=['id', 'label'],
                                    value_vars=['sentence1', 'sentence2', 'sentence3', 'sentence4'],
                                    var_name='sentence_type', value_name='text')

# Create binary labels: 1 if the sentence_type corresponds to the label, else 0
train_data_melted['binary_label'] = (train_data_melted['sentence_type'] == train_data_melted['label']).astype(int)

# Reshape the test data similarly
test_data_melted = test_data.melt(id_vars=['id'],
                                  value_vars=['sentence1', 'sentence2', 'sentence3', 'sentence4'],
                                  var_name='sentence_type', value_name='text')

In [ ]:
train_data_melted

,id,label,sentence_type,text,binary_label
0,TRAIN_000,2,sentence1,"직원들 마음에 들지 않는다는 것은 알겠지만, 가지 말아야 할까? 인터넷에서 싸게 살...",0
1,TRAIN_001,3,sentence1,분위기 최고! 2층 창문이 넓어서 공기가 통하는 느낌이에요. 조명도 멋지고 음료와 ...,0
2,TRAIN_002,2,sentence1,"일단, 장사가 잘 되길 바라는 마음에서 별 다섯 개 드립니다. 간도 딱 맞았고, 저...",0
3,TRAIN_003,4,sentence1,"1편의 독특함 때문에 살짝 뒤로 밀린 느낌이 있지만, 여전히 재미있어요. 게임 시스...",0
4,TRAIN_004,1,sentence1,"빵점 주고 싶은걸 간신히 참았다...이런건 사상 유래가 없는,조지 루카스 영감의 스...",0
...,...,...,...,...,...
195,TRAIN_045,4,sentence4,ㄹㅇ남자직원 왜케 말투 공격적인거에요? 밥먹고 기분좋게 카페와서 커피먹을라고 했는데...,0
196,TRAIN_046,3,sentence4,"이 가격에 이런 품질의 피자라니, 너무 기분이 좋아요! 도우는 매우 쫄깃하고 고구마...",0
197,TRAIN_047,2,sentence4,"겉은 바삭바삭한데, 치킨 메뉴가 많고 주인부부가 인상적이어서 들어갔는데 바로 나왔어...",0
198,TRAIN_048,4,sentence4,"단체로 와서 잘먹긴했지만 회만 먹은것도 아니고 술도 시키면서 먹고있는데 ""적당히""드...",0


----------

In [ ]:
# Constants
MODEL_NAME = 'bert-base-uncased'
MAX_LEN = 128
BATCH_SIZE = 16
EPOCHS = 3
LEARNING_RATE = 2e-5
SEED = 42

In [ ]:
# Dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels=None, tokenizer=None, max_len=MAX_LEN):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        item = {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)
        }

        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[index], dtype=torch.long)

        return item

# Model class
class TextClassifier(nn.Module):
    def __init__(self, model_name=MODEL_NAME):
        super(TextClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        _, pooled_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            return_dict=False
        )
        output = self.dropout(pooled_output)
        output = self.linear(output)
        return output

In [ ]:
# Tokenizer
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

# Data preparation with stratified splitting
X_train, X_val, y_train, y_val = train_test_split(
    train_data_melted['text'], train_data_melted['binary_label'], test_size=0.1, random_state=SEED, stratify=train_data_melted['binary_label']
)

train_dataset = TextDataset(
    texts=X_train.values,
    labels=y_train.values,
    tokenizer=tokenizer,
    max_len=MAX_LEN
)

val_dataset = TextDataset(
    texts=X_val.values,
    labels=y_val.values,
    tokenizer=tokenizer,
    max_len=MAX_LEN
)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Training function
def train_epoch(model, data_loader, optimizer, device, scheduler):
    model = model.train()
    losses = []
    correct_predictions = 0

    for d in data_loader:
        input_ids = d['ids'].to(device)
        attention_mask = d['mask'].to(device)
        token_type_ids = d['token_type_ids'].to(device)
        labels = d['labels'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )

        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, labels)

        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.double() / len(data_loader.dataset), np.mean(losses)

# Evaluation function
def eval_model(model, data_loader, device):
    model = model.eval()
    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for d in data_loader:
            input_ids = d['ids'].to(device)
            attention_mask = d['mask'].to(device)
            token_type_ids = d['token_type_ids'].to(device)
            labels = d['labels'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )

            _, preds = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, labels)

            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())

    return correct_predictions.double() / len(data_loader.dataset), np.mean(losses)


In [ ]:
# Set device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Model instantiation
model = TextClassifier(MODEL_NAME)
model = model.to(device)

# Optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
total_steps = len(train_loader) * EPOCHS
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=total_steps // 2, gamma=0.1)

# Loss function
loss_fn = nn.CrossEntropyLoss().to(device)

# Training and evaluation
history = {
    'train_acc': [],
    'train_loss': [],
    'val_acc': [],
    'val_loss': []
}

best_accuracy = 0

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(
        model,
        train_loader,
        optimizer,
        device,
        scheduler
    )

    print(f'Train loss {train_loss} accuracy {train_acc}')

    val_acc, val_loss = eval_model(
        model,
        val_loader,
        device
    )

    print(f'Val   loss {val_loss} accuracy {val_acc}')
    print()

    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)

    if val_acc > best_accuracy:
        torch.save(model.state_dict(), 'best_model_state.bin')
        best_accuracy = val_acc

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Epoch 1/3
----------
Train loss 0.3853512977560361 accuracy 0.8555555555555556
Val   loss 0.07793770730495453 accuracy 1.0

Epoch 2/3
----------
Train loss 0.048121267929673195 accuracy 1.0
Val   loss 0.02619310189038515 accuracy 1.0

Epoch 3/3
----------
Train loss 0.03073469161366423 accuracy 1.0
Val   loss 0.022570012137293816 accuracy 1.0



In [ ]:
# Prediction on test data
test_dataset = TextDataset(
    texts=test_data_melted['text'].values,
    tokenizer=tokenizer,
    max_len=MAX_LEN
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)

# Load best model
model.load_state_dict(torch.load('best_model_state.bin'))

<All keys matched successfully>

In [ ]:
# Prediction function
def predict(model, data_loader):
    model = model.eval()
    predictions = []

    with torch.no_grad():
        for d in data_loader:
            input_ids = d['ids'].to(device)
            attention_mask = d['mask'].to(device)
            token_type_ids = d['token_type_ids'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )

            _, preds = torch.max(outputs, dim=1)

            predictions.extend(preds.cpu().numpy())

    return predictions

In [ ]:
test_pred = predict(model, test_loader)
test_data_melted['label'] = test_pred

In [ ]:
sample_submission = pd.read_csv('/content/drive/MyDrive/ESAA/OB/data/ai_vs_human_text/sample_submission.csv')

In [ ]:
submission = test_data_melted[['id', 'sentence_type', 'label']]
submission['sentence_number'] = submission['sentence_type'].str.extract('(\d+)').astype(int)
final_submission = submission.loc[submission.groupby('id')['label'].idxmax()]
final_submission = final_submission[['id', 'label']].reset_index(drop=True)

In [ ]:
final_submission.to_csv('submission.csv', index=False)

In [ ]:
final_submission

,id,label
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0
...,...,...
1095,TEST_1095,0
1096,TEST_1096,0
1097,TEST_1097,0
1098,TEST_1098,0
